# Problems

## - 'In Millions / Thousands' need multiplier info conveyed to cleaning_column()

## - Dictionary holding col_name varint 

## - 'Property|equipment.'

# Capstone Project
## Structure : 
### 1) Import Libraries 
### 2) 2017 to 2020 
### 3) 2012 to 2016 

# Libraries

In [1]:
# Import Library 

import re
import requests
import unicodedata
from bs4 import BeautifulSoup
from pprint import pprint
import numpy as np
from numpy import array
import pandas as pd 
import csv
import pickle
from IPython.display import display
import pickle
from collections import Counter
from IPython.core import display as ICD

pd.set_option('display.max_columns', 100)


In [2]:
master_filings_dict = {}
        
Accession_Number_URL = {}

AN_Active = {}

name_CIK = { 
    'Microsoft' :  ['0000789019']
}

dir_url = r'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type=10-&dateb=&owner=include&count=100'

doc_url = r'https://www.sec.gov/Archives/edgar/data/{CIKx}/{xx}/{yy}.txt'

In [3]:
# Dict of Companies -> CIK
for name, CIK_list in name_CIK.items():
    
    for CIK_position, CIK in enumerate(CIK_list):
        
        # CIK -> A_N 
        new_dir_url = dir_url.format( CIK )

        new_doc_url = doc_url.format(CIKx = CIK, xx='{xx}', yy='{yy}' )

        response = requests.get( new_dir_url )
        soup = BeautifulSoup( response.content, 'lxml' )
        text = soup.get_text( strip=True )

        cleaned_text = re.findall( 'Acc-no: \d+-\d+-\d+' , text )

        accession_number = [ n.replace('Acc-no: ', '') for n in cleaned_text ]
        accessionnumber = [ num.replace('-', '') for num in accession_number ]
        
        accession_numbers = zip(accessionnumber, accession_number)

        for a,b in accession_numbers: 
            
            file_url_list = new_doc_url.format(xx = a, yy = b)

            Accession_Number_URL.update({ b : file_url_list})
            
print('Accession_Number_URL - ', '\n', Accession_Number_URL)

Accession_Number_URL -  
 {'0001564590-20-019706': 'https://www.sec.gov/Archives/edgar/data/0000789019/000156459020019706/0001564590-20-019706.txt', '0001564590-20-002450': 'https://www.sec.gov/Archives/edgar/data/0000789019/000156459020002450/0001564590-20-002450.txt', '0001564590-19-037549': 'https://www.sec.gov/Archives/edgar/data/0000789019/000156459019037549/0001564590-19-037549.txt', '0001564590-19-027952': 'https://www.sec.gov/Archives/edgar/data/0000789019/000156459019027952/0001564590-19-027952.txt', '0001564590-19-012709': 'https://www.sec.gov/Archives/edgar/data/0000789019/000156459019012709/0001564590-19-012709.txt', '0001564590-19-001392': 'https://www.sec.gov/Archives/edgar/data/0000789019/000156459019001392/0001564590-19-001392.txt', '0001564590-18-024893': 'https://www.sec.gov/Archives/edgar/data/0000789019/000156459018024893/0001564590-18-024893.txt', '0001564590-18-019062': 'https://www.sec.gov/Archives/edgar/data/0000789019/000156459018019062/0001564590-18-019062.txt

## Cleaning Columns Function


In [4]:
month_dict = {'December' : 'Q4',
              'November' : 'Q4',
             'October' : 'Q4',
             'September' : 'Q3',
             'August': 'Q3',
             'July' : 'Q3',
             'June' : 'Q2',
             'May' : 'Q2',
             'April' : 'Q2',
             'March' : 'Q1',
             'Feburary' : 'Q1',
             'January' : 'Q1'}

In [5]:
col_name_variants = {
    'Inventor' : 'Inventory',
    'Total liabilities and' : 'Total liabilities and equity',
    'Common stock.*' : 'Common Stock',
    'Preferred stock.*': 'Preferred Stock',
    'Accounts receivable.*' : 'Accounts Receivable',
    'Accrued liabilities.*' : 'Accrued Liabilities',
    'Deferred revenue.*' : 'Deferred Revenue',
    'Redeemable noncontrolling interests in subsidiaries.*' : 'Noncontrolling interests in subsidiaries',
    'Short term investments.*' : 'Short term investments', 
    'Property|equipment.' : 'Property and Equipment',
    'Retained earnings.*' : 'Retained Earnings',
    'stockholders.*' : 'Shareholder Equity',
    '^Cash.*' : 'Cash',
    'Convertible senior notes.*' : 'Convertible Senior Notes'
}

In [6]:
def clean_col_name( name ):
    
    for detect, replacement in col_name_variants.items():

        if re.findall(detect, name, flags=re.I):

            n_column = name.replace(name, replacement) 

            return n_column 
            
    return name

## Cleaning Tables Function

In [7]:
# Throw column list-like object into cleaning function. 
def cleaning_column(r_column):
     
    column_list = []
    column_name = []
    
    for element_post, element in enumerate(r_column):
        
        # Handling the first "row" in the table i.e. column name.
        if element_post == 0:

            ele_norm = unicodedata.normalize('NFKD', element)
            
            ele_norm = ele_norm.replace('net', '')
            
            ele_norm = ele_norm.replace( '-', ' ')
            
            s_ele_norm = re.sub(r"^\s+|\s+$", "", ele_norm)
            
            name = re.sub(r'(?=,|:|\.|\(){1}.*', repl='', string= s_ele_norm, flags=re.I)
            
            # clean_col_name function Choose old_if or new_dict
            clean_name = clean_col_name( name )
            ###
            column_name.append(clean_name)
            ###
            column_list.append(clean_name)
        
        # Handling the all the remaining "rows" in the table i.e. data/numbers.
        elif element not in ['\n$', '\n\xa0', '\n']:
                        
            n_element = element.replace( ',', '' )

            pattern = re.compile(r'[\d]+,[\d]+|[\d]+')

            res = pattern.findall(n_element)

            res = [int(ele) * 10**6 for ele in res]

            column_list.extend(res)

    return column_list
    

In [8]:
# New and imporved.
def Quarter( column ):
    
    print('column used for quarter', column)
    multi_list = [p_s_ele_norm.split() for p_s_ele_norm in column if p_s_ele_norm.split() != []]
#     print('multi_list', multi_list)
    new_column = [item for sublist in multi_list for item in sublist]
    print('new_column used for year', new_column)
    Q = []
    Y = []
    
    for letter in column:
#         print('letter', letter)
        try:
            month = re.findall(r'\w{}+'.format(list(month_dict.keys())), letter.replace(' ',  '') )[0]
            
            Q_num = month_dict[month] 
    #         print('Q_num' , Q_num)
    
            if Q_num != []:
                Q.append(Q_num)
                
        except:
            continue
    #         print('~'*20)
        
    print('Q' , Q) 
#     Q_final = Q[0] 
#     print('Q_final' , Q_final)
    
    Q_final =['Quarter'] + Q[:1]
    
    for letter in new_column:   
        
        #print( letter.split(',') )
        
        Year = [re.compile(r'^[0-9]{4}$').findall(p_s_ele_norm) for p_s_ele_norm in letter.split(',') if re.compile(r'^[0-9]{4}$').findall(p_s_ele_norm) != []]
        Y.append(Year)
    Y = [item for sublist in Y for item in sublist][0]
    
    Y.insert(0, 'Year')
    #Y.insert(1, 0)
    #Y.insert(1, 0)
    return Q_final, Y
    print( '\n', 'Q_final', Q_final, 'Y', Y)

In [9]:
A_N = []
for AN, url in Accession_Number_URL.items():
    
    if AN == '0001193125-12-170665':
        break
        
    else:
        A_N.append(AN)
        print(url)
        
print(A_N, '\n')
print(len(A_N))

https://www.sec.gov/Archives/edgar/data/0000789019/000156459020019706/0001564590-20-019706.txt
https://www.sec.gov/Archives/edgar/data/0000789019/000156459020002450/0001564590-20-002450.txt
https://www.sec.gov/Archives/edgar/data/0000789019/000156459019037549/0001564590-19-037549.txt
https://www.sec.gov/Archives/edgar/data/0000789019/000156459019027952/0001564590-19-027952.txt
https://www.sec.gov/Archives/edgar/data/0000789019/000156459019012709/0001564590-19-012709.txt
https://www.sec.gov/Archives/edgar/data/0000789019/000156459019001392/0001564590-19-001392.txt
https://www.sec.gov/Archives/edgar/data/0000789019/000156459018024893/0001564590-18-024893.txt
https://www.sec.gov/Archives/edgar/data/0000789019/000156459018019062/0001564590-18-019062.txt
https://www.sec.gov/Archives/edgar/data/0000789019/000156459018009307/0001564590-18-009307.txt
https://www.sec.gov/Archives/edgar/data/0000789019/000156459018001129/0001564590-18-001129.txt
https://www.sec.gov/Archives/edgar/data/0000789019

## 2017 - 2020

### Creating the .htm urls to get key. post 2016 keys are #BALANCE_SHEETS 

In [10]:
AN_Active = {}
# Get filename, method 3, use htm.# anchor to jump to Balnce sheet.
for AN, url in Accession_Number_URL.items():
    
    # Break is needed to limit lower bound, either 
    # pre-2016 = 0001193125-12-170665 (if excludes -12-170665 ) or 
    # post-16 = 0001193125-16-742796 (x > 16, excludes 16)
    
    # if statement takes in Acc-Num format: xx-xxx-xx
    if AN == '0001193125-12-170665':
        break
        
    if int( AN.split( sep='-' )[1] ) > 12:
        print('~'*75)
        r = requests.get(url)
        
        try: 
            soup = BeautifulSoup(r.content, "html")
            
        except:
            print('Look at: ')
            print( '{} - {}'.format(AN, url))
            
        # 1.0
        AccessionNumber = AN.replace('-', '')
        
        # 2.1 part of table website.
        filename = str(soup.find('filename').contents[0]).replace('\n', '')
        print('filename - ', filename, '\n')
        
        # 2.2 Anchor page_key.
        hr_tag = soup.findAll('a', href=True)
        page_key = str( [table.get('href') for table in hr_tag if 'Balance Sheets' in table.get_text()][0])
        print('page_key - ', page_key, '\n')
        
        KEY = filename + page_key
        
        active_url = new_doc_url.format( xx = AccessionNumber, yy = KEY ).replace('.txt', '')

        print('active_url - ', active_url)
        
        # Key for 2016 onwards is the same, will overwrite the keys() in AN_Active. 
        AN_Active.update({ AN : [ page_key , active_url ] } )

    else:
        continue
    

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
filename -  msft-10q_20200331.htm 

page_key -  #BALANCE_SHEETS 

active_url -  https://www.sec.gov/Archives/edgar/data/0000789019/000156459020019706/msft-10q_20200331.htm#BALANCE_SHEETS
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
filename -  msft-10q_20191231.htm 

page_key -  #BALANCE_SHEETS 

active_url -  https://www.sec.gov/Archives/edgar/data/0000789019/000156459020002450/msft-10q_20191231.htm#BALANCE_SHEETS
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
filename -  msft-10q_20190930.htm 

page_key -  #BALANCE_SHEETS 

active_url -  https://www.sec.gov/Archives/edgar/data/0000789019/000156459019037549/msft-10q_20190930.htm#BALANCE_SHEETS
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
filename -  msft-10k_20190630.htm 

page_key -  #BALANCE_SHEETS 

active_url -  https://www.sec.gov/Archives/edgar/data/0000789019/0001

In [32]:
AN_Active= {'0001564590-20-019706': ['#BALANCE_SHEETS',
  'https://www.sec.gov/Archives/edgar/data/0000789019/000156459020019706/msft-10q_20200331.htm#BALANCE_SHEETS'],
 '0001564590-20-002450': ['#BALANCE_SHEETS',
  'https://www.sec.gov/Archives/edgar/data/0000789019/000156459020002450/msft-10q_20191231.htm#BALANCE_SHEETS'],
 '0001564590-19-037549': ['#BALANCE_SHEETS',
  'https://www.sec.gov/Archives/edgar/data/0000789019/000156459019037549/msft-10q_20190930.htm#BALANCE_SHEETS'],
 '0001564590-19-027952': ['#BALANCE_SHEETS',
  'https://www.sec.gov/Archives/edgar/data/0000789019/000156459019027952/msft-10k_20190630.htm#BALANCE_SHEETS'],
 '0001564590-19-012709': ['#BALANCE_SHEETS',
  'https://www.sec.gov/Archives/edgar/data/0000789019/000156459019012709/msft-10q_20190331.htm#BALANCE_SHEETS'],
 '0001564590-19-001392': ['#BALANCE_SHEETS',
  'https://www.sec.gov/Archives/edgar/data/0000789019/000156459019001392/msft-10q_20181231.htm#BALANCE_SHEETS'],
 '0001564590-18-024893': ['#BALANCE_SHEETS',
  'https://www.sec.gov/Archives/edgar/data/0000789019/000156459018024893/msft-10q_20180930.htm#BALANCE_SHEETS'],
 '0001564590-18-019062': ['#BALANCE_SHEETS',
  'https://www.sec.gov/Archives/edgar/data/0000789019/000156459018019062/msft-10k_20180630.htm#BALANCE_SHEETS'],
 '0001564590-18-009307': ['#BALANCE_SHEETS',
  'https://www.sec.gov/Archives/edgar/data/0000789019/000156459018009307/msft-10q_20180331.htm#BALANCE_SHEETS'],
 '0001564590-18-001129': ['#BALANCE_SHEETS',
  'https://www.sec.gov/Archives/edgar/data/0000789019/000156459018001129/msft-10q_20171231.htm#BALANCE_SHEETS'],
 '0001564590-17-020171': ['#BALANCE_SHEETS',
  'https://www.sec.gov/Archives/edgar/data/0000789019/000156459017020171/msft-10q_20170930.htm#BALANCE_SHEETS'],
 '0001564590-17-014900': ['#BALANCE_SHEETS',
  'https://www.sec.gov/Archives/edgar/data/0000789019/000156459017014900/msft-10k_20170630.htm#BALANCE_SHEETS'],
 '0001564590-17-007547': ['#BALANCE_SHEETS',
  'https://www.sec.gov/Archives/edgar/data/0000789019/000156459017007547/msft-10q_20170331.htm#BALANCE_SHEETS'],
 '0001564590-17-000654': ['#BALANCE_SHEETS',
  'https://www.sec.gov/Archives/edgar/data/0000789019/000156459017000654/msft-10q_20161231.htm#BALANCE_SHEETS'],
 '0001193125-16-742796': ['#tx245252_5',
  'https://www.sec.gov/Archives/edgar/data/0000789019/000119312516742796/d245252d10q.htm#tx245252_5'],
 '0001193125-16-662209': ['#tx187868_27',
  'https://www.sec.gov/Archives/edgar/data/0000789019/000119312516662209/d187868d10k.htm#tx187868_27'],
 '0001193125-16-550254': ['#tx256147_5',
  'https://www.sec.gov/Archives/edgar/data/0000789019/000119312516550254/d256147d10q.htm#tx256147_5'],
 '0001193125-16-441821': ['#tx15167_5',
  'https://www.sec.gov/Archives/edgar/data/0000789019/000119312516441821/d15167d10q.htm#tx15167_5'],
 '0001193125-15-350718': ['#tx83804_5',
  'https://www.sec.gov/Archives/edgar/data/0000789019/000119312515350718/d83804d10q.htm#tx83804_5'],
 '0001193125-15-272806': ['#tx918813_32',
  'https://www.sec.gov/Archives/edgar/data/0000789019/000119312515272806/d918813d10k.htm#tx918813_32'],
 '0001193125-15-144151': ['#tx860721_5',
  'https://www.sec.gov/Archives/edgar/data/0000789019/000119312515144151/d860721d10q.htm#tx860721_5'],
 '0001193125-15-020351': ['#tx827041_5',
  'https://www.sec.gov/Archives/edgar/data/0000789019/000119312515020351/d827041d10q.htm#tx827041_5'],
 '0001193125-14-380252': ['#tx770268_5',
  'https://www.sec.gov/Archives/edgar/data/0000789019/000119312514380252/d770268d10q.htm#tx770268_5'],
 '0001193125-14-289961': ['#tx722626_27',
  'https://www.sec.gov/Archives/edgar/data/0000789019/000119312514289961/d722626d10k.htm#tx722626_27'],
 '0001193125-14-157088': ['#tx695543_5',
  'https://www.sec.gov/Archives/edgar/data/0000789019/000119312514157088/d695543d10q.htm#tx695543_5'],
 '0001193125-14-018634': ['#tx635047_5',
  'https://www.sec.gov/Archives/edgar/data/0000789019/000119312514018634/d635047d10q.htm#tx635047_5'],
 '0001193125-13-409855': ['#tx594455_5',
  'https://www.sec.gov/Archives/edgar/data/0000789019/000119312513409855/d594455d10q.htm#tx594455_5'],
 '0001193125-13-310206': ['#tx527745_102',
  'https://www.sec.gov/Archives/edgar/data/0000789019/000119312513310206/d527745d10k.htm#tx527745_102'],
 '0001193125-13-160748': ['#tx497226_5',
  'https://www.sec.gov/Archives/edgar/data/0000789019/000119312513160748/d497226d10q.htm#tx497226_5'],
 '0001193125-13-022207': ['#tx453668_5',
  'https://www.sec.gov/Archives/edgar/data/0000789019/000119312513022207/d453668d10q.htm#tx453668_5']}

In [12]:
print( [txt for acc_num,  txt in Accession_Number_URL.items() if 21 > int(acc_num.split(sep='-')[1]) > 16] )

['https://www.sec.gov/Archives/edgar/data/0000789019/000156459020019706/0001564590-20-019706.txt', 'https://www.sec.gov/Archives/edgar/data/0000789019/000156459020002450/0001564590-20-002450.txt', 'https://www.sec.gov/Archives/edgar/data/0000789019/000156459019037549/0001564590-19-037549.txt', 'https://www.sec.gov/Archives/edgar/data/0000789019/000156459019027952/0001564590-19-027952.txt', 'https://www.sec.gov/Archives/edgar/data/0000789019/000156459019012709/0001564590-19-012709.txt', 'https://www.sec.gov/Archives/edgar/data/0000789019/000156459019001392/0001564590-19-001392.txt', 'https://www.sec.gov/Archives/edgar/data/0000789019/000156459018024893/0001564590-18-024893.txt', 'https://www.sec.gov/Archives/edgar/data/0000789019/000156459018019062/0001564590-18-019062.txt', 'https://www.sec.gov/Archives/edgar/data/0000789019/000156459018009307/0001564590-18-009307.txt', 'https://www.sec.gov/Archives/edgar/data/0000789019/000156459018001129/0001564590-18-001129.txt', 'https://www.sec.go

In [13]:
# Post-2016 uses the following .txt links. 
Accession_Number_URL_16_20 = {'0001564590-20-019706': ['https://www.sec.gov/Archives/edgar/data/0000789019/000156459020019706/0001564590-20-019706.txt'],
 '0001564590-20-002450': ['https://www.sec.gov/Archives/edgar/data/0000789019/000156459020002450/0001564590-20-002450.txt'],
 '0001564590-19-037549': ['https://www.sec.gov/Archives/edgar/data/0000789019/000156459019037549/0001564590-19-037549.txt'],
 '0001564590-19-027952': ['https://www.sec.gov/Archives/edgar/data/0000789019/000156459019027952/0001564590-19-027952.txt'],
 '0001564590-19-012709': ['https://www.sec.gov/Archives/edgar/data/0000789019/000156459019012709/0001564590-19-012709.txt'],
 '0001564590-19-001392': ['https://www.sec.gov/Archives/edgar/data/0000789019/000156459019001392/0001564590-19-001392.txt'],
 '0001564590-18-024893': ['https://www.sec.gov/Archives/edgar/data/0000789019/000156459018024893/0001564590-18-024893.txt'],
 '0001564590-18-019062': ['https://www.sec.gov/Archives/edgar/data/0000789019/000156459018019062/0001564590-18-019062.txt'],
 '0001564590-18-009307': ['https://www.sec.gov/Archives/edgar/data/0000789019/000156459018009307/0001564590-18-009307.txt'],
 '0001564590-18-001129': ['https://www.sec.gov/Archives/edgar/data/0000789019/000156459018001129/0001564590-18-001129.txt'],
 '0001564590-17-020171': ['https://www.sec.gov/Archives/edgar/data/0000789019/000156459017020171/0001564590-17-020171.txt'],
 '0001564590-17-014900': ['https://www.sec.gov/Archives/edgar/data/0000789019/000156459017014900/0001564590-17-014900.txt'],
 '0001564590-17-007547': ['https://www.sec.gov/Archives/edgar/data/0000789019/000156459017007547/0001564590-17-007547.txt'],
 '0001564590-17-000654': ['https://www.sec.gov/Archives/edgar/data/0000789019/000156459017000654/0001564590-17-000654.txt']}

In [14]:
# AN of new anchor tag - 'id. 
print("Anchor tag chamge from 'name' to 'id' for the following Accession Number: \n")
list_AN_Active = list(AN_Active.keys())
list_AN_Active.reverse() 
name_to_id = list_AN_Active[-3:]
print(f'{name_to_id} \n')

print("Assuming all new filings are 'id' tagged.")

Anchor tag chamge from 'name' to 'id' for the following Accession Number: 

['0001564590-19-037549', '0001564590-20-002450', '0001564590-20-019706'] 

Assuming all new filings are 'id' tagged.


In [27]:

#TESTING

AN_Active = {'0001193125-16-742796' : [ '#tx245252_5' , 'https://www.sec.gov/Archives/edgar/data/0000789019/000119312516742796/d245252d10q.htm#tx245252_5' ]}

# 2016 onwards. Use the above Accession_Number_URL_16_20, 2016+ need to use .txt format. 
for AN, key_url in AN_Active.items():
    
    # Break is needed to limit lower bound, either 
    # pre-2016 = 0001193125-12-170665 (if excludes -12-170665 ) or 
    # post-16 = 0001193125-16-742796 (x > 16, excludes 16)
    print('~'*75)

    # if statement takes in Acc-Num: xx-xxx-xx
    if AN == '0001193125-12-170665':
        break

    if int( AN.split( sep='-' )[1] ) < 17:
        
        print(f'Accession Number - {AN}\n')
        key = key_url[0]
        print(f'key - {key_url[0]}\n')
        act_url = key_url[1]
        print(f'Active URL - {key_url[1]}\n')
        
        r = requests.get(act_url)
        soup = BeautifulSoup(r.content, "lxml")

        filing_document = soup.find('document')
        page_key = soup.find_all(attrs={"name": "{}".format(key.replace('#', ''))})

        document_id = filing_document.type.find(text=True, recursive=False).strip()
#         print('page_key - ', page_key)
        T_table = [ table.parent.parent.parent.next_sibling.next_sibling.next_sibling.next_sibling for table in page_key ]
        target = T_table[0].findAll('tr')
        row_list = [ row for row in target if row.get_text(strip=True, separator=',') != '']
        print(row_list)

IndentationError: expected an indented block (<ipython-input-27-c734a143e409>, line 53)

In [16]:
AN_active_test = {
    '0001564590-17-000654': ['#BALANCE_SHEETS',
  'https://www.sec.gov/Archives/edgar/data/0000789019/000156459017000654/msft-10q_20161231.htm#BALANCE_SHEETS'],
    '0001193125-16-742796': ['#tx245252_5',
  'https://www.sec.gov/Archives/edgar/data/0000789019/000119312516742796/d245252d10q.htm#tx245252_5'],}

In [34]:
Checklist ={}

# 2016 onwards. Use the above Accession_Number_URL_16_20, 2016+ need to use .txt format. 
for AN, key_url in AN_Active.items():
    
    # Break is needed to limit lower bound, either 
    # pre-2016 = 0001193125-12-170665 (if excludes -12-170665 ) or 
    # post-16 = 0001193125-16-742796 (x > 16, excludes 16)
    print('~'*75)

    # if statement takes in Acc-Num: xx-xxx-xx
    if AN == '0001193125-12-170665':
        break
    
    # For filings prior to 16, use key-tag.
    if int( AN.split( sep='-' )[1] ) < 17:
        
        print(f'Accession Number - {AN}\n')
        key = key_url[0]
        print(f'key - {key_url[0]}\n')
        act_url = key_url[1]
        print(f'Active URL - {key_url[1]}\n')
        
        r = requests.get(act_url)
        soup = BeautifulSoup(r.content, "lxml")

        filing_document = soup.find('document')
        page_key = soup.find_all(attrs={"name": "{}".format(key.replace('#', ''))})

        document_id = filing_document.type.find(text=True, recursive=False).strip()

        T_table = [ table.parent.parent.parent.next_sibling.next_sibling.next_sibling.next_sibling for table in page_key ]
#         print(T_table)
    
    # For Filings after 2016, use 'name' & 'id' anchor tags.
    if int( AN.split( sep='-' )[1] ) > 16:
        
        # Take the AN in Accession_Number_URL
        url_txt = Accession_Number_URL[AN]
        print(url_txt)
        
        r = requests.get(url_txt)
        soup = BeautifulSoup(r.content, "lxml")

        filing_document = soup.find('document')
        
        document_id = filing_document.type.find(text=True, recursive=False).strip()
        
        # For filings starting from 2020, attr: 'name' change to 'id'
        if AN in name_to_id:
            page_key = soup.find_all(attrs={"id": "BALANCE_SHEETS"})

            T_table = [table.next_sibling.next_sibling.next_sibling.next_sibling for table in page_key ]
            # print(T_table)

        else:
            page_key = soup.find_all(attrs={"name": "BALANCE_SHEETS"})

            T_table = [ page_key[0].parent.next_sibling.next_sibling.next_sibling.next_sibling ]

    target = T_table[0].findAll('tr')
    row_list = [ row for row in target if row.get_text(strip=True, separator=',') != '']

    col_list = []

    for row in row_list:
        col = row.find_all('td')
        column = []
        for tr in col:
            ele = tr.text
            
            try:
                ele= ele.encode('cp1252').decode("utf-8", "ignore")
            except:
                ele.encode('latin1').decode("utf-8", "ignore")
                
            column.append(ele)
        col_list.append(column)

    print(f'col_list[0] - {col_list[0]}')

    new_col_list = []
    for col_num, column in enumerate(col_list):
        
        if col_num == 1:
#             print('raw column - ', column)
            Q, Y = Quarter(column)
#             print(Q, Y)
            new_col_list.append(Q)
            new_col_list.append(Y)

        else:
            column.pop(1)
            column.pop(1)

            new_col_list.append(cleaning_column( column ))

    df = pd.DataFrame(new_col_list)
    df = df.T.reset_index(drop=True)
    df.columns = df.iloc[0]
    df = df.drop([row for row in range(df.shape[0]) if row != 1], axis=0).reset_index(drop=True)
    ICD.display(df)

    try: 
        with open('{}-{}.csv'.format(AN, document_id), "wb") as fp:   # Pickling
            pickle.dump(df, fp)

        # display a status to the user.
        print('All the pages from {} document {} have been tableized and saved.'.format(AN, document_id))

    except:
        Checklist.update( f'{document_id} : {AN}' )
        print("Can't save file {}, file type - {}".format(AN, document_id))
        
    print('-'*80) 

if Checklist == {}:
    print('\nEverything went well.\n')
    
else: 
    print('\nThese files have a problem.\n', Checklist)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
https://www.sec.gov/Archives/edgar/data/0000789019/000156459020019706/0001564590-20-019706.txt
col_list[0] - ['\n(In millions) (Unaudited)', '\n', '\n', '\n']
column used for quarter ['\n', '\n', '\nMarch 31,2020', '\n', '\n', '\nJune30,2019', '\n']
new_column used for year ['March', '31,2020', 'June30,2019']
Q ['Q1', 'Q2']


,,Quarter,Year,Assets,Current assets,Cash,Short term investments,Total cash,Accounts Receivable,Inventory,Other current assets,Total current assets,Property and Equipment,Operating lease right of use assets,Equity investments,Goodwill,Intangible assets,Other long term assets,Total assets,Shareholder Equity,Current liabilities,Accounts payable,Current portion of long term debt,Accrued compensation,Short term income taxes,Short term unearned revenue,Other current liabilities,Total current liabilities,Long term debt,Long term income taxes,Long term unearned revenue,Deferred income taxes,Operating lease liabilities,Other long term liabilities,Total liabilities,Commitments and contingencies,Shareholder Equity,Common Stock,Retained Earnings,Accumulated other comprehensive income,Shareholder Equity,Total liabilities and equity
0,None,Q1,2020,None,None,11710000000,125916000000,137626000000,22699000000,1644000000,8536000000,170505000000,41221000000,8448000000,2660000000,42064000000,6855000000,13696000000,285449000000,None,None,9246000000,3748000000,6254000000,3296000000,27012000000,9151000000,58707000000,62862000000,28888000000,3385000000,185000000,7248000000,9673000000,170948000000,None,None,79813000000,32012000000,2676000000,114501000000,285449000000


All the pages from 0001564590-20-019706 document 10-Q have been tableized and saved.
--------------------------------------------------------------------------------
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
https://www.sec.gov/Archives/edgar/data/0000789019/000156459020002450/0001564590-20-002450.txt
col_list[0] - ['\n(In millions) (Unaudited)', '\n', '\n', '\n']
column used for quarter ['\n', '\n', '\nDecember31,2019', '\n', '\n', '\nJune30,2019', '\n']
new_column used for year ['December31,2019', 'June30,2019']
Q ['Q4', 'Q2']


,,Quarter,Year,Assets,Current assets,Cash,Short term investments,Total cash,Accounts Receivable,Inventory,Other current assets,Total current assets,Property and Equipment,Operating lease right of use assets,Equity investments,Goodwill,Intangible assets,Other long term assets,Total assets,Shareholder Equity,Current liabilities,Accounts payable,Current portion of long term debt,Accrued compensation,Short term income taxes,Short term unearned revenue,Other current liabilities,Total current liabilities,Long term debt,Long term income taxes,Long term unearned revenue,Deferred income taxes,Operating lease liabilities,Other long term liabilities,Total liabilities,Commitments and contingencies,Shareholder Equity,Common Stock,Retained Earnings,Accumulated other comprehensive loss,Shareholder Equity,Total liabilities and equity
0,None,Q4,2019,None,None,8864000000,125389000000,134253000000,23525000000,1823000000,7473000000,167074000000,40522000000,8439000000,2755000000,42248000000,7126000000,14630000000,282794000000,None,None,8811000000,6247000000,5421000000,2687000000,27343000000,9131000000,59640000000,63361000000,28754000000,3878000000,222000000,7172000000,9658000000,172685000000,None,None,79625000000,30739000000,255000000,110109000000,282794000000


All the pages from 0001564590-20-002450 document 10-Q have been tableized and saved.
--------------------------------------------------------------------------------
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
https://www.sec.gov/Archives/edgar/data/0000789019/000156459019037549/0001564590-19-037549.txt
col_list[0] - ['\n(In millions) (Unaudited)', '\n', '\n', '\n']
column used for quarter ['\n', '\n', '\nSeptember 30,2019', '\n', '\n', '\nJune 30,2019', '\n']
new_column used for year ['September', '30,2019', 'June', '30,2019']
Q ['Q3', 'Q2']


...

All the pages from 0001193125-13-310206 document 10-K have been tableized and saved.
--------------------------------------------------------------------------------
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Accession Number - 0001193125-13-160748

key - #tx497226_5

Active URL - https://www.sec.gov/Archives/edgar/data/0000789019/000119312513160748/d497226d10q.htm#tx497226_5

col_list[0] - ['(In millions) (Unaudited)', '', '', '', '', '', '']
column used for quarter [' March31, 2013', '', '', 'June30,2012', '']
new_column used for year ['March31,', '2013', 'June30,2012']
Q ['Q1', 'Q2']


,,Quarter,Year,Assets,Current assets,Cash,Short term investments,Total cash,Accounts Receivable,Inventory,Deferred income taxes,Other,Total current assets,Property and Equipment,Equity and other investments,Goodwill,Intangible assets,Other long term assets,Total assets,Shareholder Equity,Current liabilities,Accounts payable,Current portion of long term debt,Accrued compensation,Income taxes,Short term unearned revenue,Securities lending payable,Other,Total current liabilities,Long term debt,Long term unearned revenue,Deferred income taxes,Other long term liabilities,Total liabilities,Commitments and contingencies,Shareholder Equity,Common Stock,Retained Earnings,Accumulated other comprehensive income,Shareholder Equity,Total liabilities and equity
0,None,Q1,2013,None,None,5240000000,69243000000,74483000000,11991000000,2133000000,1676000000,3241000000,93524000000,9204000000,11193000000,14682000000,3240000000,2262000000,134105000000,None,None,4532000000,2246000000,3474000000,689000000,16511000000,564000000,3913000000,31929000000,11949000000,1394000000,2424000000,9721000000,57417000000,None,None,66826000000,7657000000,2205000000,76688000000,134105000000


All the pages from 0001193125-13-160748 document 10-Q have been tableized and saved.
--------------------------------------------------------------------------------
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Accession Number - 0001193125-13-022207

key - #tx453668_5

Active URL - https://www.sec.gov/Archives/edgar/data/0000789019/000119312513022207/d453668d10q.htm#tx453668_5

col_list[0] - ['(In millions) (Unaudited)', '', '', '', '', '', '']
column used for quarter [' December31,\n2012', '  ', '', '', 'June30,2012', '']
new_column used for year ['December31,', '2012', 'June30,2012']
Q ['Q4', 'Q2']


,,Quarter,Year,Assets,Current assets,Cash,Short term investments,Total cash,Accounts Receivable,Inventory,Deferred income taxes,Other,Total current assets,Property and Equipment,Equity and other investments,Goodwill,Intangible assets,Other long term assets,Total assets,Shareholder Equity,Current liabilities,Accounts payable,Current portion of long term debt,Accrued compensation,Income taxes,Short term unearned revenue,Securities lending payable,Other,Total current liabilities,Long term debt,Long term unearned revenue,Deferred income taxes,Other long term liabilities,Total liabilities,Commitments and contingencies,Shareholder Equity,Common Stock,Retained Earnings,Accumulated other comprehensive income,Shareholder Equity,Total liabilities and equity
0,None,Q4,2012,None,None,6017000000,62295000000,68312000000,14317000000,1661000000,1983000000,3301000000,89574000000,8698000000,10707000000,14727000000,3341000000,1636000000,128683000000,None,None,4356000000,2241000000,2942000000,630000000,18354000000,21000000,3366000000,31910000000,11947000000,1459000000,2394000000,8397000000,56107000000,None,None,66334000000,4236000000,2006000000,72576000000,128683000000


All the pages from 0001193125-13-022207 document 10-Q have been tableized and saved.
--------------------------------------------------------------------------------

Everything went well.



In [36]:
new_col_list

[[''],
 ['Quarter', 'Q4'],
 ['Year', '2012'],
 ['Assets'],
 ['Current assets'],
 ['Cash', 6017000000, 6938000000],
 ['Short term investments', 62295000000, 56102000000],
 ['Total cash', 68312000000, 63040000000],
 ['Accounts Receivable', 14317000000, 15780000000],
 ['Inventory', 1661000000, 1137000000],
 ['Deferred income taxes', 1983000000, 2035000000],
 ['Other', 3301000000, 3092000000],
 ['Total current assets', 89574000000, 85084000000],
 ['Property and Equipment', 8698000000, 8269000000],
 ['Equity and other investments', 10707000000, 9776000000],
 ['Goodwill', 14727000000, 13452000000],
 ['Intangible assets', 3341000000, 3170000000],
 ['Other long term assets', 1636000000, 1520000000],
 ['Total assets', 128683000000, 121271000000],
 ['Shareholder Equity'],
 ['Current liabilities'],
 ['Accounts payable', 4356000000, 4175000000],
 ['Current portion of long term debt', 2241000000, 1231000000],
 ['Accrued compensation', 2942000000, 3875000000],
 ['Income taxes', 630000000, 789000000]

##### 2017- 2020 working material

In [18]:
type(column[0])

str

In [19]:
url = 'https://www.sec.gov/Archives/edgar/data/0000789019/000156459019012709/0001564590-19-012709.txt'
r = requests.get(url)
soup = BeautifulSoup(r.content, "html.parser")

page_key = soup.find_all(attrs={"name": "BALANCE_SHEETS"})
    
T_table = [page_key[0].parent.next_sibling.next_sibling.next_sibling.next_sibling ]
target = T_table[0].findAll('tr')
print(target)

[<tr>
<td style="width:70%;" valign="bottom">
<p style="text-align:left;margin-bottom:0pt;margin-top:0pt;margin-left:0pt;;text-indent:0pt;;font-weight:bold;font-size:7.5pt;font-family:Arial;font-style:normal;text-transform:none;font-variant: normal;">(In millions) (Unaudited)</p></td>
<td style="width:1%;" valign="bottom">
<p style="margin-bottom:0pt;margin-top:0pt;margin-left:0pt;;text-indent:0pt;;font-size:7.5pt;font-family:Arial;font-weight:normal;font-style:normal;text-transform:none;font-variant: normal;"> </p></td>
<td colspan="6" style="width:28%;" valign="bottom">
<p style="margin-bottom:0pt;margin-top:0pt;margin-left:0pt;;text-indent:0pt;;font-family:Arial;font-size:4pt;font-weight:normal;font-style:normal;text-transform:none;font-variant: normal;"> </p></td>
<td style="width:1%;" valign="bottom">
<p style="margin-bottom:0pt;margin-top:0pt;margin-left:0pt;;text-indent:0pt;;font-size:7.5pt;font-family:Arial;font-weight:normal;font-style:normal;text-transform:none;font-variant: 

In [20]:
page_key

[<a name="BALANCE_SHEETS"></a>, <a name="BALANCE_SHEETS"></a>]

In [21]:
print([row for row in range(df.shape[0]) if row != 1])

[0]


In [22]:
[table.next_sibling.next_sibling.next_sibling.next_sibling for table in page_key ]

AttributeError: 'NoneType' object has no attribute 'next_sibling'

In [ ]:
T_table = [ table.next_sibling.next_sibling.children for table in page_key ]
print(T_table)

## End 2017-2020

In [ ]:
# Show Accession_Number_URL
Accession_Number_URL

## 2012 -2016

### Creating the .htm urls to get key.

In [ ]:
AN_Active = {}
# Get filename, method 3, use htm.# anchor to jump to Balnce sheet.
for AN, url in Accession_Number_URL.items():
    
    # Break is needed to limit lower bound, either 
    # pre-2016 = 0001193125-12-170665 (if excludes -12-170665 ) or 
    # post-16 = 0001193125-16-742796 (x > 16, excludes 16)
    
    # if statement takes in Acc-Num: xx-xxx-xx
    if AN == '0001193125-12-170665':
        break
        
    if int( AN.split( sep='-' )[1] ) < 17:
        print('~'*75)
        r = requests.get(url)
        
        try: 
            soup = BeautifulSoup(r.content, "html")
            
        except:
            print('Look at: ')
            print( '{} - {}'.format(AN, url))
            
        # 1.0
        AccessionNumber = AN.replace('-', '')
        
        # 2.1 part of table website.
        filename = str(soup.find('filename').contents[0]).replace('\n', '')
        print('filename - ', filename, '\n')
        
        # 2.2 Anchor page_key.
        hr_tag = soup.findAll('a', href=True)
        page_key = str( [table.get('href') for table in hr_tag if 'Balance Sheets' in table.get_text()][0])
        print('page_key - ', page_key, '\n')
        
        KEY = filename + page_key
        
        active_url = new_doc_url.format( xx = AccessionNumber, yy = KEY ).replace('.txt', '')

        print('active_url - ', active_url)
        
        # Key for 2016 onwards is the same, will overwrite the keys() in AN_Active. 
        AN_Active.update({ page_key : [active_url, AN]})

    else:
        continue
    

##### 2012- 2016 working material

AN_Active = {
 '0001193125-16-742796': ['https://www.sec.gov/Archives/edgar/data/0000789019/000119312516742796/0001193125-16-742796.txt'],
 '0001193125-16-662209': ['https://www.sec.gov/Archives/edgar/data/0000789019/000119312516662209/0001193125-16-662209.txt'],
 '0001193125-16-550254': ['https://www.sec.gov/Archives/edgar/data/0000789019/000119312516550254/0001193125-16-550254.txt'],
 '0001193125-16-441821': ['https://www.sec.gov/Archives/edgar/data/0000789019/000119312516441821/0001193125-16-441821.txt'],
 '0001193125-15-350718': ['https://www.sec.gov/Archives/edgar/data/0000789019/000119312515350718/0001193125-15-350718.txt'],
 '0001193125-15-272806': ['https://www.sec.gov/Archives/edgar/data/0000789019/000119312515272806/0001193125-15-272806.txt'],
 '0001193125-15-144151': ['https://www.sec.gov/Archives/edgar/data/0000789019/000119312515144151/0001193125-15-144151.txt'],
 '0001193125-15-020351': ['https://www.sec.gov/Archives/edgar/data/0000789019/000119312515020351/0001193125-15-020351.txt'],
 '0001193125-14-380252': ['https://www.sec.gov/Archives/edgar/data/0000789019/000119312514380252/0001193125-14-380252.txt'],
 '0001193125-14-289961': ['https://www.sec.gov/Archives/edgar/data/0000789019/000119312514289961/0001193125-14-289961.txt'],
 '0001193125-14-157088': ['https://www.sec.gov/Archives/edgar/data/0000789019/000119312514157088/0001193125-14-157088.txt'],
 '0001193125-14-018634': ['https://www.sec.gov/Archives/edgar/data/0000789019/000119312514018634/0001193125-14-018634.txt'],
 '0001193125-13-409855': ['https://www.sec.gov/Archives/edgar/data/0000789019/000119312513409855/0001193125-13-409855.txt'],
 '0001193125-13-310206': ['https://www.sec.gov/Archives/edgar/data/0000789019/000119312513310206/0001193125-13-310206.txt'],
 '0001193125-13-160748': ['https://www.sec.gov/Archives/edgar/data/0000789019/000119312513160748/0001193125-13-160748.txt'],
 '0001193125-13-022207': ['https://www.sec.gov/Archives/edgar/data/0000789019/000119312513022207/0001193125-13-022207.txt']
}

In [ ]:
# Show AN_Active
AN_Active

In [ ]:
# Before 2017 AND After 0001193125-12-170665
for key, url_AN in AN_Active.items():
            
    AN = url_AN[1]
    url = url_AN[0]
    
    print('key - ', key)
    print('\n')
    print('Accession Number - ', AN)
    print('\n')
    print('Active URL - ', url)
    print('\n')
    
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")

    filing_document = soup.find('document')
    page_key = soup.find_all(attrs={"name": "{}".format(key.replace('#', ''))})
    
    document_id = filing_document.type.find(text=True, recursive=False).strip()
        
    T_table = [ table.parent.parent.parent.next_sibling.next_sibling.next_sibling.next_sibling for table in page_key ]    
    # print('T_table - ', T_table)
    
    target = T_table[0].findAll('tr')
    row_list = [ row for row in target if row.get_text(strip=True, separator=',') != '']

    col_list = []

    for row in row_list:
        col = row.find_all('td')
        column = []
        for tr in col:
            ele = tr.text
            ele= ele.encode('latin1').decode("utf-8", "ignore")
            column.append(ele)
        col_list.append(column)

    col_list.pop(0)

    new_col_list = []
    for col_num, column in enumerate(col_list): 

        if col_num == 0:
            print(column)
            Q, Y = Quarter(column)
            print(Q, Y)
            new_col_list.append(Q)
            new_col_list.append(Y)

        else:
            new_col_list.append(cleaning_column(column))

    df = pd.DataFrame(new_col_list)
    
    df = df.T.reset_index(drop=True)
    df.columns = df.iloc[0]
    df = df.drop([0,2], axis=0).reset_index(drop=True)
    ICD.display(df)

    try: 
        with open('{}-{}.csv'.format(AN, document_id), "wb") as fp:   # Pickling
            pickle.dump(df, fp)

        # display a status to the user.
        print('All the pages from {} document {} have been tableized and saved.'.format(AN, document_id))

    except:
        value_list.append(vv)
        print("Can't save file {}, file type = {}".format(AN, document_id))

        print('-'*80) 


# New Method try 2016 and before, reference material ->

In [ ]:
# Got the key 
test_url = 'https://www.sec.gov/Archives/edgar/data/0000789019/000119312516662209/d187868d10k.htm#tx187868_27'

r = requests.get(test_url)
soup = BeautifulSoup(r.content, "html")

hr_tag = soup.findAll('a', href=True)
list_of_table_in_page = [table.get('href') for table in hr_tag if 'Balance Sheets' in table.get_text() ][0]
print(list_of_table_in_page)

page_key = soup.find_all(attrs={"name": "tx187868_27"})
# add this to table below -> .findAll('tr')
T_table = [ table.parent.parent.parent.next_sibling.next_sibling.next_sibling.next_sibling for table in page_key ]
print(T_table)

# Figuring out how to use the key.

In [ ]:
def has_key(name):
    if  re.compile("#tx245252_5").search(name):
        return name 

In [ ]:
test_url = 'https://www.sec.gov/Archives/edgar/data/0000789019/000119312516742796/0001193125-16-742796.txt'

r = requests.get(test_url)
soup = BeautifulSoup(r.content, "lxml")

page_key = soup.find_all(attrs={"name": "tx245252_5"})

T_table = [ table.parent.parent.parent.next_sibling.next_sibling.next_sibling.next_sibling for table in page_key ]

In [ ]:
page_key

In [ ]:
target = T_table[0].findAll('tr')
print(target)

In [ ]:
type(target)

In [ ]:
row_list = [ row for row in target if row.get_text(strip=True, separator=',') != '']
print(row_list)

In [ ]:
col_list = []
for row in row_list:
    col = row.find_all('td')
    column = []
    for tr in col:
        ele = tr.get_text()
        ele= ele.encode('latin1').decode('cp1252')
        column.append(ele)
    col_list.append(column)
    
print(col_list)

In [ ]:
col_list.pop(0)

In [ ]:
new_col_list = []
for col_num, column in enumerate(col_list): 
    
    if col_num == 0:
        print(column)
        Q, Y = Quarter(column)
        print(Q, Y)
        new_col_list.append(Q)
        new_col_list.append(Y)
        
    else:
        new_col_list.append(cleaning_column(column))

In [ ]:
new_col_list

In [ ]:
df = pd.DataFrame(new_col_list)
df = df.T.reset_index(drop=True)
df

In [ ]:
df.columns = df.iloc[0]
df = df.drop([0,2], axis=0).reset_index(drop=True)


## End of old reference material 

# < TESTING AREA>

In [ ]:
# url = 'https://www.sec.gov/Archives/edgar/data/0000789019/000156459020019706/msft-10q_20200331.htm#BALANCE_SHEETS'
url = 'https://www.sec.gov/Archives/edgar/data/0000789019/000156459020019706/0001564590-20-019706.txt'
key = '0001564590-20-019706'

In [ ]:
r = requests.get(url)
soup = BeautifulSoup(r.content, "lxml")

filing_document = soup.find('document')
document_id = filing_document.type.find(text=True, recursive=False).strip()

if key in list(Accession_Number_URL_16_20.keys())[0:3]:
    page_key = soup.find_all(attrs={"id": "BALANCE_SHEETS"})

    T_table = [table.next_sibling.next_sibling.next_sibling.next_sibling for table in page_key ]
    # print(T_table)

else:
    page_key = soup.find_all(attrs={"name": "BALANCE_SHEETS"})

    T_table = [ page_key[0].parent.next_sibling.next_sibling.next_sibling.next_sibling ]

target = T_table[0].findAll('tr')
row_list = [ row for row in target if row.get_text(strip=True, separator=',') != '']

col_list = []

for row in row_list:
    col = row.find_all('td')
    column = []
    for tr in col:
        ele = tr.text
        ele= ele.encode('cp1252').decode("utf-8", "ignore")
        column.append(ele)
    col_list.append(column)

col_list.pop(0)

new_col_list = []
for col_num, r_column in enumerate(col_list): 

    if col_num == 0:
        print('raw column - ', r_column)
        Q, Y = Quarter(r_column)
        print(Q, Y)
        new_col_list.append(Q)
        new_col_list.append(Y)

    else:
        
        new_col_list.append(cleaning_column( r_column ))

df = pd.DataFrame(new_col_list)
df = df.T.reset_index(drop=True)
df.columns = df.iloc[0]
df = df.drop([row for row in range(df.shape[0]) if row != 1], axis=0).reset_index(drop=True)
ICD.display(df)


In [ ]:
new_dict = [name for name in df.columns]
print(new_dict, len(new_dict))

In [ ]:
old_if = [name for name in df.columns]
print(old_if, len(old_if))

In [ ]:
import collections
print([item for item, count in collections.Counter(old_if).items() if count > 1])


In [ ]:
print(set(old_if) - set(new_dict), '\n')
print(set(new_dict) - set(old_if))

# </END TESTING AREA>

#                     {Nexted Function Territory}
## Take care of the multiplier in cleaning_column() and else where
## Limit: slow, not lean, ineffcient, relied heavily on indices (position in a list).

In [ ]:
QorK = {'10-Q' : [],
        '10-K' : []
       }

for acc_num, url in Accession_Number_URL.items():
    
    master_document_dict = {}
    
    # create a stop point
    if acc_num == '0001193125-16-742796':
        break

    else:
        
        # grab the response
        response = requests.get(url[0])

    # Soupify
        # pass it through the parser, in this case let's just use lxml because the tags seem to follow xml.
        soup = BeautifulSoup(response.content, 'lxml')
        filing_document = soup.find('document')
        
    # Parsing
        # Document type ->> document_id
        document_id = filing_document.type.find(text=True, recursive=False).strip()
        # Document sequence ->> document_sequence
        document_sequence = filing_document.sequence.find(text=True, recursive=False).strip()
        # Document filename ->> document_filename
        document_filename = filing_document.filename.find(text=True, recursive=False).strip()
        # Document description ->> document_description
        document_description = filing_document.description.find(text=True, recursive=False).strip()
        
    # Storage    
        # initalize our document dictionary
        master_document_dict[document_id] = {}

        # add the different parts, we parsed up above.
        master_document_dict[document_id]['document_sequence'] = document_sequence
        master_document_dict[document_id]['document_filename'] = document_filename
        master_document_dict[document_id]['document_description'] = document_description
        
# Scraping
        # grab the text portion of the document, this will be used to split the document into pages.
        filing_doc_text = filing_document.find('text').extract()

        # find all the thematic breaks, these help define page numbers and page breaks.
        all_thematic_breaks = filing_doc_text.find_all('table')

        # Locate and store page number via list comprehension.
        all_page_numbers = [thematic_break.previous_sibling for thematic_break in all_thematic_breaks if len(str(thematic_break))>6]
            
        # convert all thematic breaks to a string so it can be used for parsing
        all_thematic_breaks = [str(thematic_break) for thematic_break in all_thematic_breaks]

        # prep the document text for splitting, this means converting it to a string.
        filing_doc_string = str(filing_doc_text)
    
        # handle the case where there are thematic breaks.
        if len(all_thematic_breaks) > 0: 

            # define the regex delimiter pattern, this would just be all of our thematic breaks.
            regex_delimiter_pattern = '|'.join(map(re.escape, all_thematic_breaks))

            # split the document along each thematic break.
            split_filing_string = re.split(regex_delimiter_pattern, filing_doc_string)

            # store the document itself
            master_document_dict[document_id]['pages_code'] = split_filing_string

        # handle the case where there are no thematic breaks.
        elif len(all_thematic_breaks) == 0:

            # handles so it will display correctly.
            split_filing_string = all_thematic_breaks

            # store the document as is, since there are no thematic breaks. In other words, no splitting.
            master_document_dict[document_id]['pages_code'] = [filing_doc_string]
            
        # display some information to the user.
        print('-'*80)
        print('The document {} was parsed.'.format(document_id))
        print('There was {} page(s) found.'.format(len(all_page_numbers)))
        print('There was {} thematic breaks(s) found.'.format(len(all_thematic_breaks)))

        # store the documents in the master_filing_dictionary.
        master_filings_dict[acc_num]['filing_documents'] = master_document_dict
        
        # if document is 10-Q
        if document_id == '10-Q':
            QorK['10-Q'].append(acc_num) # add acc_num to QorK in 10-Q as key
        
        # if document is 10-K
        if document_id == '10-K':
            QorK['10-K'].append(acc_num) # add acc_num to QorK in 10-K as key
        
        del master_document_dict
        
        print('-'*80)
        print('All the documents for filing {} were parsed and stored.'.format(acc_num))
    


In [ ]:
string_test = master_filings_dict['0001193125-16-742796']['filing_documents']['10-Q']['pages_code']
for page_num, page in enumerate(string_test):
    if ('Cash and cash equivalents' in page) and ('Short-term investments' in page):
        print( string_test.index(page_num  + 3) )
        print('~'*40)
#string_test[56]
# df01 = table(string_test, '10-Q')
# df = df01[0]
# df

## Anchor Check Function



In [ ]:
# Variations of HTML anchors.
CBS_A = 'BALANCE SHEETS'
CBS_B = 'name="BALANCE_SHEETS"'
CBS_D = 'BALANCE SHEETS '
CBS_C = 'Cash and cash equivalents'
# SoO_A = '<a name="Statements_of_Operations"' 
# SoO_B = 'id="CONSOLIDATED_STATEMENTS_OPERATIONS"'
# SoO3 = 'name="CONSOLIDATED_STATEMENTS_OPERATIONS"'
# CSoO10K = 'name="Consolidated_Statements_of_Operations"'
# CSoO10Knew = 'id="Consolidated_Statements_of_Operations"'

# CF_A = '<a name="Statements_of_Cash'
# CF_B = 'id="CONSOLIDATED_STATEMENTS_CASH_FLOWS"'
# CF3 = 'name="CONSOLIDATED_STATEMENTS_CASH_FLOWS"'
# CCF10K = 'name="Consolidated_Statements_of_Cash_Flows"'
# CCF10Knew = 'id="Consolidated_Statements_of_Cash_Flows"'

def anchor_check(page_code):
    
    # if the page has one of these anchors
    if (CBS_A in page_code) or (CBS_B in page_code) or (CBS_D in page_code) and (CBS_C in page_code):
        return True

    #elif (SoO_A in page_code) or (SoO_B in page_code) or (SoO3 in page_code) or (CSoO10K in page_code) or (CSoO10Knew in page_code) :
        #return True

    #elif (CF_A in page_code) or (CF_B in page_code) or (CF3 in page_code) or (CCF10K in page_code) or (CCF10Knew in page_code):
        #return True


    else:
        return False


## Cleaning Columns and Cleaning Tables

In [ ]:
def table_extractor( page, multiplier, k ):
    
    # Convert string to BS object
    soup = BeautifulSoup(page, 'html5')

    # then get all the rows in the table.
    table_rows = soup.find_all('tr')
    
    if len( [tr.text for tr in table_rows[0].find_all('td')] ) < 3:
        del table_rows[0]
    
    single_table = []
    Date_column = []
    
    # Loop through each column, adding in three structures to single_table.
    for tr_post, tr in enumerate(table_rows):
        
        td = tr.find_all('td')

        column = [tr.text for tr in td]
        #print( 'tr_post', tr_post, len(column), column)
        # 10-Q 
        if k == '10-Q':
            
            if tr_post == 3:

                # Quarter Column
                col_Q, col_year = Quarter( column )
                single_table.append( col_Q )
                # Year Column
                single_table.append( col_year )

            elif tr_post > 4: 

                # Features Columns
                cleaned_data = cleaning_column( column , multiplier )
                if len(cleaned_data) > 10:
                    del cleaned_data[3]
                    
                single_table.append( cleaned_data )
                
        # 10-K
        elif k == '10-K':
            
#             if tr_post == 1 or tr_post == 2:
                    
#                 Date_column.append(column)
                
            if tr_post == 3:
                
                # Quarter Column
                col_date, col_year = Quarter(column)
                single_table.append( col_date )
                # Year Column
                single_table.append( col_year )
                
            elif tr_post > 4:              
                
                cleaned_data = cleaning_column( column , 10**6 )
                if len(cleaned_data) > 10:
                    del cleaned_data[3]
                
                single_table.append( cleaned_data )
            
    return single_table

In [ ]:
def df_clean( single_table ):
    print(single_table)
    table_df = pd.DataFrame(single_table)

    table_df = table_df.transpose()  
    print(table_df)
    table_df.drop(table_df.index[1:3], 0, inplace=True)

    #table_df.drop(table_df.index[2:], 0, inplace=True)

    table_df.columns = table_df.iloc[0]

    table_df = table_df.drop(table_df.index[0])
    
    return table_df

In [ ]:
def table(page_code, k):
    
    doc_tables=[]

    for page_post, page in enumerate(page_code):
        
        if ('(In millions) (Unaudited)' in page) and ('BALANCE SHEETS ' in page):
            
            single_table = table_extractor( page, 10**6, k )
            #print(single_table)
            table_df = df_clean(single_table)
                        
            doc_tables.append(table_df)
        
        elif ('BALANCE SHEETS ' in page) and ('(In thousands) (Unaudited)' in page):
            
            #print('page_post', page_post)
            
            single_table = table_extractor( page, 10**3, k )

            table_df = df_clean(single_table)

            doc_tables.append(table_df)
            
        else:
            print(page_post+1, 'anchor_check / in thousands/million problem')

                
    return doc_tables
    

# Exploring the Data

In [ ]:
explore_dict = {}
for QK, AccNum_list in QorK.items():
    
    #for an_postition, acc_num in enumerate(AccNum_list): 
    if QK== '10-Q':
        Q_new = AccNum_list[0]
        Q_old = AccNum_list[-1]
        explore_dict[QK] = [Q_new, Q_old]

    if QK == '10-K':
        K_new = AccNum_list[0]
        K_old = AccNum_list[-1]
        explore_dict[QK] = [K_new, K_old]

In [ ]:
QorK ={'10-Q': ['0001564590-20-019706',
  '0001564590-20-002450',
  '0001564590-19-037549',
  '0001564590-19-012709',
  '0001564590-19-001392',
  '0001564590-18-024893',
  '0001564590-18-009307',
  '0001564590-18-001129',
  '0001564590-17-020171',
  '0001564590-17-007547',
  '0001564590-17-000654'],
 '10-K': ['0001564590-19-027952',
  '0001564590-18-019062',
  '0001564590-17-014900']}

In [ ]:
explore_dict

In [ ]:
for QK, N_O in explore_dict.items():
    
    for an_postition, acc_num in enumerate(N_O):
        
        pages = master_filings_dict[acc_num]['filing_documents'][QK]['pages_code']
        print('\n', 'This is {} a {} document.'.format(acc_num, QK))
        test_df_list = table(pages, QK)
        test_df = test_df_list[0]
        
        table_df = test_df.drop(test_df.index[1:]).reset_index(drop=True)
        print('Shape - ', table_df.shape)
        ICD.display(table_df)
        

In [ ]:
Quarter(['\n\xa0', '\n\xa0', '\nMarch 31,2020', '\n\xa0', '\n\xa0', '\nJune\xa030,2019', '\n\xa0'])

# Save Files Locally. 

In [ ]:
Checklist = {}

for k, v in QorK.items():
    value_list = []
    for i, vv in enumerate(v):

        # first grab 10-Q documents only
        pages = master_filings_dict[v[i]]['filing_documents'][k]['pages_code']

        try:
            # Extract Table from html code and wrap it as a list of df.
            test_df_list = table(pages, k)
            test_df = test_df_list[0]
            table_df = test_df.drop(test_df.index[1:]).reset_index(drop=True)
            #ICD.display(table_df[0])

            with open('{}-{}.csv'.format(v[i], k), "wb") as fp:   # Pickling
                pickle.dump(table_df, fp)

            # display a status to the user.
            print('All the pages from {} document {} have been tableized and saved.'.format(k, v[i]))
        
        except: 
            value_list.append(vv)
            print('Something went wrong in file {}, file type = {}'.format(v[i], k))
        
        print('-'*80) 
    Checklist['{}'.format(k)] = value_list
    del value_list

In [ ]:
# Checklist of erroneous filings.  
for a, b in Checklist.items():
    if b != []:
        print('These have a problem.', '\n', a, b)

    else: 
        print(a, 'has no problem!! :) ' , '\n')
    
print(Checklist)

In [ ]:
# Bring QorK over to Part 2. 
QorK